# B. Process from the XLSX files

- **3:citations** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/3_2025_March 9th_citation_counts.xlsm`
  -  il s’agit des citations : je ne l’ai pas travaillé.  
 - **4:first&last** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/4_working first and last.xlsx`
    -   ils’agit d’une liste de 20 auteurs qui sont first and last and mais ne pas prendre en compte ceux qui sont PI.  

In [1]:
import pandas as pd
import pickle
import preprocess_utils
import numpy as np
import json
import plot_info

def clean_authors_claims(df):
    df['Sex'] = df['Sex'].map({1: 'Male', 0: 'Female', '??': np.nan, '?': np.nan})
    df = df.drop(columns=[col for col in df.columns if '%' in col])
    df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
    df = df.rename(columns={'Conituinity': 'Continuity', 
                                                "Partially verified":"Partially Verified", 
                                                "*Historical lab after 1998":"Historical lab after 1998"
                                                })
    if "Historical lab" in df.columns:
        df['Historical lab'] = df['Historical lab'].astype('boolean')
    if "Continuity" in df.columns:
        df['Continuity'] = df['Continuity'].astype('boolean')
    if "Become a Pi" in df.columns:
        df.loc[df["Become a Pi"] == "??", "Become a Pi"] = np.nan
        df.loc[df["Become a Pi"] == 0, "Become a Pi"] = False
        df.loc[df["Become a Pi"] == 1, "Become a Pi"] = True
    if "MD" in df.columns:
        df.loc[df["MD"] == "??", "MD"] = np.nan
        df.loc[df["MD"] == "?", "MD"] = np.nan
        df.loc[df["MD"] == 'O', "MD"] = False
        df.loc[df["MD"] == 0, "MD"] = False
        df.loc[df["MD"] == 1, "MD"] = True
    if "Ivy league" in df.columns:
        df.loc[df["Ivy league"] == 0, "Ivy league"] = False
        df.loc[df["Ivy league"] == 1, "Ivy league"] = True
    if "PhD Post-doc" in df.columns:
        df.loc[df["PhD Post-doc"] == "pi", "PhD Post-doc"] = "PI"
        df.loc[df["PhD Post-doc"] == "Pi", "PhD Post-doc"] = "PI"
    df = df.dropna(subset=['Name'])
    # drop  Schneider DS+D from names as  Schneider DS and  Schneider D are also there in separate rows
    df = df[~(df['Name'] == 'Schneider DS+D')]
    return df


# Load the database stuff
authors_db = pd.read_csv("preprocessed_data/authors_db.csv")
authors_db.sort_values("author_key")


claims_db = pd.read_csv("preprocessed_data/claims_db.csv")
major_claims_df = claims_db[claims_db['assertion_type'] == 'major_claim']


In [2]:
major_claims_df

,id,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,...,last_author_extracted,primary_affiliation,country,assertion_type,label,is_assessed,assessment_type,rank_assessment_type,shangai_ranking_2010,assessment_type_grouped
8,577,#gene:FBgn0000279 is immune inducible and is p...,3103.0,2,NaN,Tryselius Y;Samakovlis C;Kimbrell DA;Hultmark D,"CecC, a cecropin gene expressed during metamor...",1740152.0,204,1,...,Hultmark D,"Department of Microbiology, Stockholm Universi...",Sweden,major_claim,Major claim,True,Verified,1.0,79.0,Verified
30,5845,Hemocyte proliferation and lamellocyte formati...,2204.0,3,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Hultmark D,"Umeå Centre for Molecular Pathogenesis, Byggna...",Sweden,major_claim,Major claim,True,Verified,1.0,201.0,Verified
31,5841,The Ras/MAPK pathway is a main regulator of th...,2204.0,1,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Hultmark D,"Umeå Centre for Molecular Pathogenesis, Byggna...",Sweden,major_claim,Major claim,True,Verified,1.0,201.0,Verified
37,5843,The JAK/STAT and JNK pathways are major regula...,2204.0,2,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Hultmark D,"Umeå Centre for Molecular Pathogenesis, Byggna...",Sweden,major_claim,Major claim,True,Verified,1.0,201.0,Verified
110,545,#gene:FBgn0035976 functions upstream of #gene:...,2520.0,2,NaN,Gottar M;Gobert V;Michel T;Belvin M;Duyk G;Hof...,The Drosophila immune response against Gram-ne...,11912488.0,416,6881,...,Royet J,UPR 9022 du Centre National de la Recherche Sc...,France,major_claim,Major claim,True,Verified,1.0,NaN,Verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13261,9082,Ehrlichia chaffeensis establishes a pathogenic...,1593.0,1,NaN,Luce-Fedrow A;Von Ohlen T;Chapes SK,Ehrlichia chaffeensis infections in Drosophila...,19687202.0,77,11,...,Chapes SK,"Kansas State University, Division of Biology, ...",USA,major_claim,Major claim,True,"Unchallenged, logically consistent",6.0,301.0,Unchallenged
13264,9728,B. bassiana inhibits PO activation by preventi...,1532.0,2,NaN,Matskevich AA;Quintin J;Ferrandon D,The Drosophila PRR GNBP3 assembles effector co...,20201042.0,40,5,...,Ferrandon D,"Equipe Fondation Recherche Médicale, UPR 9022 ...",France,major_claim,Major claim,True,"Unchallenged, logically consistent",6.0,NaN,Unchallenged
13270,1769,"A polyclonal antibody against mammalian IRF-1,...",3004.0,2,NaN,Georgel P;Kappler C;Langley E;Gross I;Nicolas ...,Drosophila immunity. A sequence homologous to ...,7537872.0,23,7,...,Hoffmann JA,Institut de Biologie Moléculaire et Cellulaire...,France,major_claim,Major claim,True,"Unchallenged, logically inconsistent",7.0,NaN,Unchallenged
13273,2906,A pathway in addition to Toll regulates degrad...,2874.0,3,NaN,Wu LP;Anderson KV,Regulated nuclear import of Rel proteins in th...,9510254.0,392,6671,...,Anderson KV,"Molecular Biology Program, Memorial Sloan-Kett...",USA,major_claim,Major claim,True,"Unchallenged, logically inconsistent",7.0,NaN,Unchallenged


In [3]:
authors_db[authors_db["author_key"].str.contains("promislow")]

,author_id,article_id,name,leading_author,first_author,career_stage,expertise_level,author_key
640,659,1669,Promislow DE,True,False,Junior PI,Newcomer,promislow de
699,718,1921,Promislow DE,True,False,Junior PI,Newcomer,promislow de
736,716,1848,Promislow DE,True,False,Junior PI,Newcomer,promislow de
746,768,1945,Promislow DE,True,False,Senior PI,Newcomer,promislow de


## B.1. Read the stat author xlsx files
- **0: First** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/0_First author cleaned_March 18th.xlsx`
  - Pour les statistiques sur ceux qui deviennent PI, Il ne faut pas prendre en compte ceux qui sont surlignés en bleu car ils sont des premiers auteurs qui sont déjà PI. Normalement ce fichier est bon.  
Avec un peu d’approximation, je trouve que sur 291 first authors: 107 become PI et il y  16  ?? (indéterminés)et 13 qui sont déjà PI en bleu :  ce qui fait 107/262=>40,8% (262=291-16-13)  
- **2: Both** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/2_2025_March 9th_stats_author.xlsx`
  - obtenu depuis le site
  -  tu trouveras les last avec de nouveau critères  pour historical (tradition), continuty, first and last.  (pour junior et senior il faut voir par articles dans la database)  
  - Feuille 1 : Parmi les first author (1er feuille) :  
    - Zhenting Zhang et Zhaolin Zhang  sont fusionnés en Zhang Z  
    - Zhipeng Wang et Zhi Wang sont fusionné en Wang Z  
    - Hedengren M et Hedengren M et Hedengren-Olcott M ne sont pas fusionnés
  - Feuille 3: pour les last authors:
    - Il faut fusionner Schneider DS et Schneider D -> c'est les mêmes
    - Historical lab (tradition) or not : il faut pendre le colonne Historical lab after 1998 et comparer 1 (historical, trained in hustorical lab) et 0 (non historical) mais ne pas prendre en compte 2.
    - La colonne E first and Last : compte les chercheurs qui apparaissent comme first author  et last authors. Il faudrait comparer 1 (last author qui ont été first author dans une autre labo mais pas PI)  à  0 (last author withouth being first author)  sans prendre en compte les 2 (first author who were already PI when first author).
  

In [4]:
# read an xlx file in pandas
input_file_folder = "input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th"

stat_author_fn = f"{input_file_folder}/2_2025_March 9th_stats_author.xlsx"

first_authors_claims = pd.read_excel(stat_author_fn, sheet_name='First')
leading_authors_claims = pd.read_excel(stat_author_fn, sheet_name='Leading')
leading_authors_claims["Authorship"]= "Leading"
first_authors_claims["Authorship"]= "First"

first_authors_claims = clean_authors_claims(first_authors_claims)
leading_authors_claims = clean_authors_claims(leading_authors_claims)

leading_authors_claims = preprocess_utils.build_author_key(leading_authors_claims, author_name_col="Name", key_col="leading_author_key")
# # remove the one with name "Name" Promislow D  as promislow DE is there and has the right information
leading_authors_claims = leading_authors_claims[~(leading_authors_claims['leading_author_key'] == 'promislow d')]

first_authors_claims = preprocess_utils.build_author_key(first_authors_claims, author_name_col="Name", key_col="first_author_key")

In [5]:
first_authors_claims

,Name,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Authorship,first_author_key
0,Williams MJ,Male,5,14,7,3,1,0,3,First,williams mj
1,Lemaitre B,Male,4,12,0,11,1,0,0,First,lemaitre b
2,Ha EM,Female,4,10,3,1,2,2,2,First,ha em
3,Leulier F,Male,4,10,0,10,0,0,0,First,leulier f
4,De Gregorio E,Male,3,9,1,8,0,0,0,First,de gregorio e
...,...,...,...,...,...,...,...,...,...,...,...
287,Schlenke TA,Male,1,1,0,1,0,0,0,First,schlenke ta
288,Uttenweiler-Joseph S,Female,1,1,0,1,0,0,0,First,uttenweiler-joseph s
289,Wang LN,Female,1,1,0,1,0,0,0,First,wang ln
290,Yuan Y,Male,1,1,0,1,0,0,0,First,yuan y


In [6]:
leading_authors_claims

,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Start lab,Finish,Authorship,leading_author_key
0,Hultmark D,True,2.0,True,2.0,Male,28.0,67.0,12.0,47.0,6.0,0.0,2.0,H,Retired,Leading,hultmark d
1,Lemaitre B,True,1.0,True,1.0,Male,21.0,60.0,7.0,47.0,5.0,0.0,1.0,1998,still active,Leading,lemaitre b
2,Hoffmann JA,True,2.0,True,2.0,Male,16.0,45.0,2.0,39.0,2.0,0.0,2.0,H,Retired,Leading,hoffmann ja
4,Lee WJ,False,0.0,True,0.0,Male,12.0,31.0,8.0,14.0,3.0,2.0,4.0,2000,still active,Leading,lee wj
5,Ip YT,False,0.0,False,1.0,Male,11.0,30.0,5.0,21.0,3.0,1.0,0.0,1998,2020,Leading,ip yt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Nappi AJ,True,2.0,True,0.0,Male,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,Leading,nappi aj
154,Palli SR,False,0.0,False,0.0,Male,1.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,Leading,palli sr
155,Seroude L,False,0.0,False,0.0,Male,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,Leading,seroude l
162,Schneider DS,False,0.0,True,2.0,Male,12.0,31.0,17.0,6.0,1.0,0.0,7.0,NaN,NaN,Leading,schneider ds


In fact, first_authors_claims is useless here as everything is in the `first_authors_covar` file. However, leading_authors_claims has precious information 

In [7]:
first_authors_covar_fn = f"{input_file_folder}/0_First author cleaned_March 18th.xlsx"

highlighted_blue = pd.DataFrame(["Gateff E", "Hultmark D", "Imler JL", "Kappler C", "Kurucz E", 
                                "Lanot R", "Meister M", "Nappi AJ", "Reichhart JM", "RIZKI MT", 
                                "Rizki RM", "Schneider DS", "Wegner KM"], columns=["Name"])
highlighted_blue = preprocess_utils.build_author_key(highlighted_blue, author_name_col="Name", key_col="first_author_key")

first_authors_covar = pd.read_excel(first_authors_covar_fn)
first_authors_covar["Name"] = first_authors_covar["AAA_first author"]
first_authors_covar["Become a Pi"] = first_authors_covar["AAA_Become a Pi"]
first_authors_covar = first_authors_covar.drop(columns=["AAA_first author", "AAA_Become a Pi","Source"])

first_authors_covar = clean_authors_claims(first_authors_covar)
print(first_authors_covar["Sex"].unique(), 
        first_authors_covar["MD"].unique(), 
        first_authors_covar["Become a Pi"].unique(), 
        first_authors_covar["Ivy league"].unique())


# Check for duplicates in Name
duplicated_keys = first_authors_covar[first_authors_covar["Name"].duplicated(keep=False)]
# Sort by Name to see duplicates together
duplicated_keys = duplicated_keys.sort_values("Name")
# Print the result
print(f"Found {len(duplicated_keys)} rows with duplicate keys")
print(duplicated_keys[["Name", "Become a Pi", "Sex"]])

# These are put manually int he xlsx file:
first_authors_covar.loc[262, "Name"] = "Wang Zhipeng"
first_authors_covar.loc[263, "Name"] = "Wang Zhi"
first_authors_covar.loc[285, "Name"] = "Zhang Zhenting"
first_authors_covar.loc[286, "Name"] = "Zhang Zhaolin"
first_authors_covar = preprocess_utils.build_author_key(first_authors_covar, author_name_col="Name", key_col="first_author_key")

first_authors_covar["ignore_for_become_a_pi"] = False
for aut_key in highlighted_blue["first_author_key"]:
    first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "ignore_for_become_a_pi"] = True


# Create a binary column "Is a PI" based on the "PhD Post-doc" column
first_authors_covar["Is a PI"] = first_authors_covar["PhD Post-doc"] == "PI"

first_authors_covar["Career Stage"] = first_authors_covar["PhD Post-doc"].copy()
first_authors_covar["PhD Post-doc"] = first_authors_covar["PhD Post-doc"].map(
    lambda x: "PhD" if x == "PhD" else ("Post-doc" if x == "Post-doc" else pd.NA)
)

first_authors_covar["Become a PI that counts"] = False

first_authors_covar.loc[ 
    (first_authors_covar["Become a Pi"] == True) & 
    (first_authors_covar["ignore_for_become_a_pi"] == False), "Become a PI that counts"] = True
# important that these are NA as they cannot become a PI
first_authors_covar.loc[ 
    (first_authors_covar["Become a Pi"] == True) & 
    (first_authors_covar["ignore_for_become_a_pi"] == True), "Become a PI that counts"] = pd.NA

first_authors_covar = first_authors_covar.drop(columns=["ignore_for_become_a_pi", "Become a Pi"])
first_authors_covar = first_authors_covar.rename(columns={"Become a PI that counts": "Become a PI"})
first_authors_covar = first_authors_covar.rename(
    columns={
            "Sex": "First Author Sex",
            "MD": "First Author MD",
            "Current Job": "First Author Current Job",
            "Career Stage": "First Author Career Stage",
            "Is a PI": "First Author Is a PI",
            "Name": "First Author Name",
            "Become a PI": "First Author Become a PI",})
first_authors_covar

['Male' 'Female' nan] [False nan True] [True False nan] [False True]
Found 4 rows with duplicate keys
        Name Become a Pi     Sex
262   Wang Z       False    Male
263   Wang Z         NaN  Female
285  Zhang Z       False  Female
286  Zhang Z       False  Female


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_97870/333310170.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["Ivy league"] == 0, "Ivy league"] = False
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_97870/1849615998.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  first_authors_covar.loc[


,First Author Sex,PhD Post-doc,current job,First Author MD,Affiliation,Country,Ivy league,First Author Name,first_author_key,First Author Is a PI,First Author Career Stage,First Author Become a PI
0,Male,<NA>,PI,False,Forschungszentrum Borstel,Germany,False,Abdelsadik A,abdelsadik a,False,Senior Staff,True
1,Male,Post-doc,PI,False,Harvard Medical School,USA,True,Agaisse H,agaisse h,False,Post-doc,True
2,Female,PhD,Industry,False,University of Massachusetts Medical School,USA,False,Aggarwal K,aggarwal k,False,PhD,False
3,NaN,PhD,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,Ao J,ao j,False,PhD,False
4,Male,Post-doc,PI,False,Harvard Medical School and Massachusetts Gener...,USA,True,Apidianakis Y,apidianakis y,False,Post-doc,True
...,...,...,...,...,...,...,...,...,...,...,...,...
285,Female,PhD,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,Zhang Zhenting,zhang zhenting,False,PhD,False
286,Female,PhD,Admin,False,University of Kentucky,USA,False,Zhang Zhaolin,zhang zhaolin,False,PhD,False
287,Female,PhD,??,False,University of California San Diego,USA,True,Zhao HW,zhao hw,False,PhD,False
288,Male,PhD,??,False,University of Houston,USA,False,Zhou Z,zhou z,False,PhD,False


## B.2 Build First Author tidy-ish dataframe

author_db that are first_author with first_authors_covar

and _then_ claims with that

In [8]:
len(authors_db[authors_db["first_author"]]["author_key"].unique())

292

In [9]:
first_authors_db = authors_db[authors_db["first_author"]].copy()
first_authors_db

,author_id,article_id,name,leading_author,first_author,career_stage,expertise_level,author_key
0,2,2053,Brun S,False,True,Post-doc,Experienced,brun s
2,1,1650,Charroux B,False,True,Post-doc,Experienced,charroux b
4,5,2278,Silverman N,False,True,Post-doc,Newcomer,silverman n
6,7,3437,Basset A,False,True,PhD,Experienced,basset a
9,11,3096,Reichhart JM,False,True,Senior PI,Experienced,reichhart jm
...,...,...,...,...,...,...,...,...
775,490,2009,Minakhina S,False,True,Post-doc,Experienced,minakhina s
784,790,1763,Gordon MD,False,True,Master,Newcomer,gordon md
785,791,2113,Gordon MD,False,True,PhD,Experienced,gordon md
786,481,2572,Tauszig S,False,True,PhD,Experienced,tauszig s


In [10]:

first_author_all = pd.merge(first_authors_db, first_authors_covar, how="left", left_on="author_key", right_on="first_author_key", suffixes=("_db", "_covar"))
print(len(first_authors_db.author_key.unique()), len(first_authors_covar.first_author_key.unique()), len(first_author_all.author_key.unique()))

292 290 292


In [11]:
#first_authors_covar.sort_values("first_author_key").first_author_key.unique()

In [12]:
first_author_all[first_author_all["First Author Name"].isna()]

,author_id,article_id,name,leading_author,first_author,career_stage,expertise_level,author_key,First Author Sex,PhD Post-doc,current job,First Author MD,Affiliation,Country,Ivy league,First Author Name,first_author_key,First Author Is a PI,First Author Career Stage,First Author Become a PI
30,70,2239,Hedengren-Olcott M,False,True,Post-doc,Experienced,hedengren-olcott m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,338,2311,Pearson AM,False,True,Post-doc,Experienced,pearson am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,365,3447,Rizki TM,True,True,Senior PI,Experienced,rizki tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,363,3448,Rizki TM,True,True,Senior PI,Experienced,rizki tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,364,3292,Rizki TM,True,True,Senior PI,Experienced,rizki tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,362,3273,Rizki TM,True,True,Post-doc,Experienced,rizki tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
first_author_all

,author_id,article_id,name,leading_author,first_author,career_stage,expertise_level,author_key,First Author Sex,PhD Post-doc,current job,First Author MD,Affiliation,Country,Ivy league,First Author Name,first_author_key,First Author Is a PI,First Author Career Stage,First Author Become a PI
0,2,2053,Brun S,False,True,Post-doc,Experienced,brun s,Male,Post-doc,Academia,False,CNRS_Gif-sur-Yvette_Université Paris-Saclay,France,False,Brun S,brun s,False,Post-doc,False
1,1,1650,Charroux B,False,True,Post-doc,Experienced,charroux b,Male,<NA>,Senior Staff,False,CNRS University of Marseille,France,False,Charroux B,charroux b,False,Senior Staff,False
2,5,2278,Silverman N,False,True,Post-doc,Newcomer,silverman n,Male,Post-doc,PI,False,University of Massachusetts Medical School,USA,True,Silverman N,silverman n,False,Post-doc,True
3,7,3437,Basset A,False,True,PhD,Experienced,basset a,Male,PhD,Academia,False,CNRS_Gif-sur-Yvette_Université Paris-Saclay,France,False,Basset A,basset a,False,PhD,False
4,11,3096,Reichhart JM,False,True,Senior PI,Experienced,reichhart jm,Male,<NA>,Retired,False,CNRS-University of Strasbourg,France,False,Reichhart JM,reichhart jm,True,PI,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,490,2009,Minakhina S,False,True,Post-doc,Experienced,minakhina s,Female,Post-doc,??,False,Rutgers University,USA,False,Minakhina S,minakhina s,False,Post-doc,False
396,790,1763,Gordon MD,False,True,Master,Newcomer,gordon md,Male,PhD,PI,False,Stanford University School of Medicine,USA,True,Gordon MD,gordon md,False,PhD,True
397,791,2113,Gordon MD,False,True,PhD,Experienced,gordon md,Male,PhD,PI,False,Stanford University School of Medicine,USA,True,Gordon MD,gordon md,False,PhD,True
398,481,2572,Tauszig S,False,True,PhD,Experienced,tauszig s,Female,PhD,PI,False,CNRS-University of Strasbourg,France,False,Tauszig S,tauszig s,False,PhD,True


In [14]:
# merge the claims:
first_author_all = pd.merge(major_claims_df, first_author_all, how="left", left_on="article_id", right_on="article_id", suffixes=("_claim_db", "_aut"))
first_author_all

,id,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,...,current job,First Author MD,Affiliation,Country,Ivy league,First Author Name,first_author_key,First Author Is a PI,First Author Career Stage,First Author Become a PI
0,577,#gene:FBgn0000279 is immune inducible and is p...,3103.0,2,NaN,Tryselius Y;Samakovlis C;Kimbrell DA;Hultmark D,"CecC, a cecropin gene expressed during metamor...",1740152.0,204,1,...,PI,False,University of Stockholm,Sweden,False,Tryselius Y,tryselius y,False,PhD,False
1,5845,Hemocyte proliferation and lamellocyte formati...,2204.0,3,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Industry,False,University of Stockholm,Sweden,False,Zettervall CJ,zettervall cj,False,PhD,False
2,5841,The Ras/MAPK pathway is a main regulator of th...,2204.0,1,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Industry,False,University of Stockholm,Sweden,False,Zettervall CJ,zettervall cj,False,PhD,False
3,5843,The JAK/STAT and JNK pathways are major regula...,2204.0,2,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Industry,False,University of Stockholm,Sweden,False,Zettervall CJ,zettervall cj,False,PhD,False
4,545,#gene:FBgn0035976 functions upstream of #gene:...,2520.0,2,NaN,Gottar M;Gobert V;Michel T;Belvin M;Duyk G;Hof...,The Drosophila immune response against Gram-ne...,11912488.0,416,6881,...,Industry,False,CNRS University of Marseille,France,False,Gottar M,gottar m,False,PhD,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,9082,Ehrlichia chaffeensis establishes a pathogenic...,1593.0,1,NaN,Luce-Fedrow A;Von Ohlen T;Chapes SK,Ehrlichia chaffeensis infections in Drosophila...,19687202.0,77,11,...,??,False,Kansas State University,USA,False,Luce-Fedrow A,luce-fedrow a,False,PhD,False
1002,9728,B. bassiana inhibits PO activation by preventi...,1532.0,2,NaN,Matskevich AA;Quintin J;Ferrandon D,The Drosophila PRR GNBP3 assembles effector co...,20201042.0,40,5,...,??,False,CNRS-University of Strasbourg,France,False,Matskevich AA,matskevich aa,False,PhD,False
1003,1769,"A polyclonal antibody against mammalian IRF-1,...",3004.0,2,NaN,Georgel P;Kappler C;Langley E;Gross I;Nicolas ...,Drosophila immunity. A sequence homologous to ...,7537872.0,23,7,...,PI,False,CNRS-University of Strasbourg,France,False,Georgel P,georgel p,False,Post-doc,True
1004,2906,A pathway in addition to Toll regulates degrad...,2874.0,3,NaN,Wu LP;Anderson KV,Regulated nuclear import of Rel proteins in th...,9510254.0,392,6671,...,PI,False,Memorial Sloan-Kettering Cancer Center and the...,USA,True,Wu LP,wu lp,False,Post-doc,True


In [15]:
first_author_all["expertise_level"].unique()

array(['Experienced', 'Newcomer'], dtype=object)

In [16]:
first_author_all = first_author_all.apply(preprocess_utils.safe_strip)
# Map assessment types to standardized categories
first_author_all['assessment_type_grouped'] = first_author_all['assessment_type'].apply(plot_info.group_assessment)
first_author_all["First Author Career Stage"] = first_author_all["First Author Career Stage"].replace("??", "ND")
first_author_all["First Author Career Stage"] = first_author_all["First Author Career Stage"].replace("Tech", "Technician")
first_author_all.to_csv("preprocessed_data/first_author_claims.csv", index=False)

## B.3 Build Last Author tidy-ish dataframe

author_db that are first_author with first_authors_covar

and _then_ claims with that

In [17]:
authors_db

,author_id,article_id,name,leading_author,first_author,career_stage,expertise_level,author_key
0,2,2053,Brun S,False,True,Post-doc,Experienced,brun s
1,3,2053,Lemaitre B,True,False,Junior PI,Experienced,lemaitre b
2,1,1650,Charroux B,False,True,Post-doc,Experienced,charroux b
3,97,2149,Söderhäll K,True,False,Senior PI,Experienced,soderhall k
4,5,2278,Silverman N,False,True,Post-doc,Newcomer,silverman n
...,...,...,...,...,...,...,...,...
792,797,2067,Cerenius L,True,False,Senior PI,Experienced,cerenius l
793,798,2122,Higgins DE,True,False,Senior PI,Newcomer,higgins de
794,799,2184,Mengin-Lecreulx D,True,False,Senior PI,Experienced,mengin-lecreulx d
795,800,2130,Moore KJ,True,False,Junior PI,Newcomer,moore kj


In [18]:
leading_authors_covar = leading_authors_claims.drop(columns=plot_info.assessment_columns + ["Articles", "Major claims"])
leading_authors_covar.loc[leading_authors_covar["F and L"] == 0, "F and L"] = False
leading_authors_covar.loc[leading_authors_covar["F and L"] == 1, "F and L"] = True
leading_authors_covar.loc[leading_authors_covar["F and L"] == 2, "F and L"] = pd.NA

# same for Historical lab after 1998
leading_authors_covar.loc[leading_authors_covar["Historical lab after 1998"] == 0, "Historical lab after 1998"] = False
leading_authors_covar.loc[leading_authors_covar["Historical lab after 1998"] == 1, "Historical lab after 1998"] = True
leading_authors_covar.loc[leading_authors_covar["Historical lab after 1998"] == 2, "Historical lab after 1998"] = pd.NA


leading_authors_covar = leading_authors_covar.rename(columns={
            "Name": "Leading Author Name",
            "Sex": "Leading Author Sex",})

/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_97870/1090608553.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  leading_authors_covar.loc[leading_authors_covar["F and L"] == 0, "F and L"] = False
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_97870/1090608553.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  leading_authors_covar.loc[leading_authors_covar["Historical lab after 1998"] == 0, "Historical lab after 1998"] = False


In [19]:
leading_authors_covar.loc[leading_authors_covar["Leading Author Name"] == "Ehret-Sabatier L", "Continuity"]

76    True
Name: Continuity, dtype: boolean

In [20]:
# Data correction, from bruno on Apri 24th, 2025:
leading_authors_covar.loc[leading_authors_covar["leading_author_key"] == "silverman n", "Historical lab after 1998"] = False
#  Bruno : Je considere maintenant les historical lab comme : 
# K .Söderhall, J. Hoffmann, J.-M. Reichhart, D. Hultmark, H. Steiner, TM Rizki, S. Natori, A. Nappi, I Faye and E. Gateff.
update_to_continuity = ["Badenhorst P", "Boccard F", "Fauvarque MO", "Gay NJ"]
for aut_key in update_to_continuity:
    print( leading_authors_covar.loc[leading_authors_covar["Leading Author Name"] == aut_key, "Continuity"])
    leading_authors_covar.loc[leading_authors_covar["Leading Author Name"] == aut_key, "Continuity"] = True

leading_authors_covar.loc[leading_authors_covar["Leading Author Name"] == "Ehret-Sabatier L", "Continuity"] = False

49    False
Name: Continuity, dtype: boolean
68    False
Name: Continuity, dtype: boolean
29    False
Name: Continuity, dtype: boolean
22    False
Name: Continuity, dtype: boolean


In [21]:
leading_authors_covar

,Leading Author Name,Historical lab,Historical lab after 1998,Continuity,F and L,Leading Author Sex,Start lab,Finish,Authorship,leading_author_key
0,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
1,Lemaitre B,True,True,True,True,Male,1998,still active,Leading,lemaitre b
2,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
4,Lee WJ,False,False,True,False,Male,2000,still active,Leading,lee wj
5,Ip YT,False,False,False,True,Male,1998,2020,Leading,ip yt
...,...,...,...,...,...,...,...,...,...,...
153,Nappi AJ,True,<NA>,True,False,Male,NaN,NaN,Leading,nappi aj
154,Palli SR,False,False,False,False,Male,NaN,NaN,Leading,palli sr
155,Seroude L,False,False,False,False,Male,NaN,NaN,Leading,seroude l
162,Schneider DS,False,False,True,<NA>,Male,NaN,NaN,Leading,schneider ds


In [22]:
len(authors_db[authors_db["leading_author"]]["author_key"].unique())

156

In [23]:
leading_authors_covar[leading_authors_covar["leading_author_key"].str.contains("promislow")]

,Leading Author Name,Historical lab,Historical lab after 1998,Continuity,F and L,Leading Author Sex,Start lab,Finish,Authorship,leading_author_key
45,Promislow DE,False,False,False,False,Male,2006,2010,Leading,promislow de


In [24]:
leading_authors_db = authors_db[authors_db["leading_author"]].copy()
leading_author_all = pd.merge(leading_authors_db, leading_authors_covar, how="left", 
                        left_on="author_key", right_on="leading_author_key", suffixes=("_db", "_covar"))
print(len(leading_authors_db.author_key.unique()), len(leading_authors_covar.leading_author_key.unique()), len(leading_author_all.author_key.unique()))

156 156 156


In [25]:
leading_author_all[leading_author_all["author_key"] == "hoffmann ja"]

,author_id,article_id,name,leading_author,first_author,career_stage,expertise_level,author_key,Leading Author Name,Historical lab,Historical lab after 1998,Continuity,F and L,Leading Author Sex,Start lab,Finish,Authorship,leading_author_key
5,12,3096,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
9,22,3015,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
15,44,2593,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
19,58,3052,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
65,141,3141,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
66,143,2891,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
83,179,3011,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
90,195,2945,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
91,199,2978,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
100,203,2849,Hoffmann JA,True,False,Senior PI,Experienced,hoffmann ja,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja


In [26]:
# merge the claims:
leading_author_all = pd.merge(major_claims_df, leading_author_all, how="left", left_on="article_id", right_on="article_id", suffixes=("_claim_db", "_aut"))
leading_author_all

,id,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,...,Leading Author Name,Historical lab,Historical lab after 1998,Continuity,F and L,Leading Author Sex,Start lab,Finish,Authorship,leading_author_key
0,577,#gene:FBgn0000279 is immune inducible and is p...,3103.0,2,NaN,Tryselius Y;Samakovlis C;Kimbrell DA;Hultmark D,"CecC, a cecropin gene expressed during metamor...",1740152.0,204,1,...,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
1,5845,Hemocyte proliferation and lamellocyte formati...,2204.0,3,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
2,5841,The Ras/MAPK pathway is a main regulator of th...,2204.0,1,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
3,5843,The JAK/STAT and JNK pathways are major regula...,2204.0,2,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Hultmark D,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d
4,545,#gene:FBgn0035976 functions upstream of #gene:...,2520.0,2,NaN,Gottar M;Gobert V;Michel T;Belvin M;Duyk G;Hof...,The Drosophila immune response against Gram-ne...,11912488.0,416,6881,...,Royet J,True,True,True,False,Male,1999,still active,Leading,royet j
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,9082,Ehrlichia chaffeensis establishes a pathogenic...,1593.0,1,NaN,Luce-Fedrow A;Von Ohlen T;Chapes SK,Ehrlichia chaffeensis infections in Drosophila...,19687202.0,77,11,...,Chapes SK,False,False,False,False,Male,NaN,NaN,Leading,chapes sk
1002,9728,B. bassiana inhibits PO activation by preventi...,1532.0,2,NaN,Matskevich AA;Quintin J;Ferrandon D,The Drosophila PRR GNBP3 assembles effector co...,20201042.0,40,5,...,Ferrandon D,True,True,True,True,Male,1999,still active,Leading,ferrandon d
1003,1769,"A polyclonal antibody against mammalian IRF-1,...",3004.0,2,NaN,Georgel P;Kappler C;Langley E;Gross I;Nicolas ...,Drosophila immunity. A sequence homologous to ...,7537872.0,23,7,...,Hoffmann JA,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja
1004,2906,A pathway in addition to Toll regulates degrad...,2874.0,3,NaN,Wu LP;Anderson KV,Regulated nuclear import of Rel proteins in th...,9510254.0,392,6671,...,Anderson KV,False,False,False,False,Female,1997,2010,Leading,anderson kv


In [27]:
leading_author_all["career_stage"].unique()

array(['Senior PI', 'Junior PI', 'Post-doc', 'Master'], dtype=object)

In [28]:
leading_author_all["Junior Senior"] = leading_author_all["career_stage"].map(
    lambda x: "Junior PI" if x == "Junior PI" else ("Senior PI" if x == "Senior PI" else pd.NA)
)

# check taht we don't bring back the bug with the bad indexing
assert (leading_author_all[leading_author_all['leading_author_key'] == "hoffmann ja"]["Junior Senior"] == leading_author_all[leading_author_all['leading_author_key'] == "hoffmann ja"]["career_stage"]).all()

In [29]:
leading_author_all = leading_author_all.apply(preprocess_utils.safe_strip)

# Map assessment types to standardized categories
leading_author_all['assessment_type_grouped'] = leading_author_all['assessment_type'].apply(plot_info.group_assessment)


leading_author_all.to_csv("preprocessed_data/leading_author_claims.csv", index=False)

In [30]:
leading_author_all

,id,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,...,Historical lab,Historical lab after 1998,Continuity,F and L,Leading Author Sex,Start lab,Finish,Authorship,leading_author_key,Junior Senior
0,577,#gene:FBgn0000279 is immune inducible and is p...,3103.0,2,NaN,Tryselius Y;Samakovlis C;Kimbrell DA;Hultmark D,"CecC, a cecropin gene expressed during metamor...",1740152.0,204,1,...,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d,Senior PI
1,5845,Hemocyte proliferation and lamellocyte formati...,2204.0,3,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d,Senior PI
2,5841,The Ras/MAPK pathway is a main regulator of th...,2204.0,1,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d,Senior PI
3,5843,The JAK/STAT and JNK pathways are major regula...,2204.0,2,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,True,<NA>,True,<NA>,Male,H,Retired,Leading,hultmark d,Senior PI
4,545,#gene:FBgn0035976 functions upstream of #gene:...,2520.0,2,NaN,Gottar M;Gobert V;Michel T;Belvin M;Duyk G;Hof...,The Drosophila immune response against Gram-ne...,11912488.0,416,6881,...,True,True,True,False,Male,1999,still active,Leading,royet j,Junior PI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,9082,Ehrlichia chaffeensis establishes a pathogenic...,1593.0,1,NaN,Luce-Fedrow A;Von Ohlen T;Chapes SK,Ehrlichia chaffeensis infections in Drosophila...,19687202.0,77,11,...,False,False,False,False,Male,NaN,NaN,Leading,chapes sk,Senior PI
1002,9728,B. bassiana inhibits PO activation by preventi...,1532.0,2,NaN,Matskevich AA;Quintin J;Ferrandon D,The Drosophila PRR GNBP3 assembles effector co...,20201042.0,40,5,...,True,True,True,True,Male,1999,still active,Leading,ferrandon d,Senior PI
1003,1769,"A polyclonal antibody against mammalian IRF-1,...",3004.0,2,NaN,Georgel P;Kappler C;Langley E;Gross I;Nicolas ...,Drosophila immunity. A sequence homologous to ...,7537872.0,23,7,...,True,<NA>,True,<NA>,Male,H,Retired,Leading,hoffmann ja,Senior PI
1004,2906,A pathway in addition to Toll regulates degrad...,2874.0,3,NaN,Wu LP;Anderson KV,Regulated nuclear import of Rel proteins in th...,9510254.0,392,6671,...,False,False,False,False,Female,1997,2010,Leading,anderson kv,Senior PI
